In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [2]:
tf.test.is_gpu_available()

True

In [2]:
batchsize = 15

In [20]:
# train model

In [21]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 0.2, shear_range = 0.2, zoom_range = 0.2, width_shift_range =0.2, height_shift_range = 0.2, validation_split = 0.2)

In [22]:
train_data = train_datagen.flow_from_directory(r'D:\python\projects\internship\CV_Driver_Drowsiness_Detection\MRL Eye Data\Prepeared Data\train',
                                              target_size = (80,80), batch_size = batchsize, class_mode = 'categorical', subset = 'training')

validation_data = train_datagen.flow_from_directory(r'D:\python\projects\internship\CV_Driver_Drowsiness_Detection\MRL Eye Data\Prepeared Data\train',
                                              target_size = (80,80), batch_size = batchsize, class_mode = 'categorical', subset = 'validation')

Found 61497 images belonging to 2 classes.
Found 15373 images belonging to 2 classes.


In [23]:
# test model

In [24]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_data = test_datagen.flow_from_directory(r'D:\python\projects\internship\CV_Driver_Drowsiness_Detection\MRL Eye Data\Prepeared Data\test',
                                              target_size = (80,80), batch_size = batchsize, class_mode = 'categorical')

Found 8028 images belonging to 2 classes.


In [25]:
# model generation - base model

In [26]:
bmodel = VGG16(include_top = False, weights = 'imagenet', input_tensor = Input(shape = (80,80,3), batch_size = batchsize))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'relu')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [27]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(15, 80, 80, 3)]         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (15, 80, 80, 64)          1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (15, 80, 80, 64)          36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (15, 40, 40, 64)          0         
_________________________________________________________________
block2_conv1 (Conv2D)        (15, 40, 40, 128)         73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (15, 40, 40, 128)         147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (15, 20, 20, 128)         0   

In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

In [29]:
checkpoint = ModelCheckpoint(r'D:\python\projects\internship\CV_Driver_Drowsiness_Detection\models\model.h5',
                            monitor = 'val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience = 7, verbose = 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor = 'val_loss', patience = 3, verbose = 3)

callbacks = [checkpoint,earlystop,learning_rate]

In [30]:
!pip install pillow

In [31]:
from PIL.Image import core as _imaging

In [32]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,epochs=10)

Epoch 1/10
4098/4099 [============================>.] - ETA: 1s - loss: 0.3415 - accuracy: 0.8484
Epoch 00001: val_loss improved from inf to 0.50030, saving model to D:\python\projects\internship\CV_Driver_Drowsiness_Detection\models\model.h5
4099/4099 [==============================] - 6012s 1s/step - loss: 0.3415 - accuracy: 0.8484 - val_loss: 0.5003 - val_accuracy: 0.7242
Epoch 2/10
4098/4099 [============================>.] - ETA: 1s - loss: 0.2771 - accuracy: 0.8828
Epoch 00002: val_loss did not improve from 0.50030
4099/4099 [==============================] - 6420s 2s/step - loss: 0.2771 - accuracy: 0.8828 - val_loss: 0.5311 - val_accuracy: 0.7076
Epoch 3/10
4098/4099 [============================>.] - ETA: 2s - loss: 0.2639 - accuracy: 0.8918
Epoch 00003: val_loss did not improve from 0.50030
4099/4099 [==============================] - 10520s 3s/step - loss: 0.2638 - accuracy: 0.8918 - val_loss: 0.5037 - val_accuracy: 0.7403
Epoch 4/10
4098/4099 [============================>.]

In [33]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

0.20468240316025912
0.91809356


In [34]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

0.6249875994571825
0.724517


In [36]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_test)
print(loss_test)

0.32417492160898087
0.83495265
